In [ ]:
import pandas as pd

# Define the paths to your CSV files
csv_file_path_1 = 'data.csv'   # Replace with your actual first file path
csv_file_path_2 = 'AEcodiert240430_UTF8.csv'  # Replace with your actual second file path

# Read the first CSV file into a DataFrame
try:
    df1 = pd.read_csv(csv_file_path_1, delimiter=';', encoding='utf-8')
except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path_1}' not found.")
    exit()

# Read the second CSV file into a DataFrame
try:
    df2 = pd.read_csv(csv_file_path_2, delimiter=';', encoding='utf-8')
except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path_2}' not found.")
    exit()

# Display the first few rows of the concatenated DataFrame to verify
print("First few rows of the concatenated DataFrame:")
print(df1.head())  # Display the first few rows of the concatenated DataFrame

# Display the DataFrame to verify
print("DataFrame from CSV file:")
print(df2.head())

print("Columns in DataFrame:")
print(df1.columns)
print(df2.columns)

df = pd.DataFrame(df2)

In [ ]:
import pandas as pd

def split_and_expand_df(df):
  """Splits rows with multiple llt_codes into separate rows, preserving descriptions.

  Args:
    df: The input DataFrame.

  Returns:
    The expanded DataFrame.
  """

  def split_row(row):
    llt_code_list = str(row['llt_code']).split(',')
    return pd.DataFrame({'llt_code': llt_code_list, 'ae_description': row['ae_description']})

  # Apply split_row to each row using apply
  df = df.apply(split_row, axis=1)

  # Concatenate resulting DataFrames
  df = pd.concat(df.tolist(), ignore_index=True)

  return df

splitData = pd.DataFrame(df)
expanded_df = split_and_expand_df(splitData.copy())

print(expanded_df)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

feature = expanded_df.llt_code
print('Feature: ')
print(feature)

# Create multiclass one-hot encoder
one_hot_multiclass = MultiLabelBinarizer()

# One-hot encode multiclass feature
one_hot_multiclass.fit_transform(feature)

print(one_hot_multiclass.classes_)

In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

MAXLEN = 192

def preprocess_text(data):
    """ take texts and prepare as input features for BERT 
    """
    input_ids = []
    # For every sentence...
    for comment in data:
        encoded_sent = tokenizer.encode_plus(
            text=comment,
            add_special_tokens=True,  # Add `[CLS]` and `[SEP]`
            max_length=MAXLEN,  # Max length to truncate/pad
            pad_to_max_length=True,  # Pad sentence to max length
            return_attention_mask=False,  # attention mask not needed for our task
        )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get("input_ids"))
    return input_ids

input_ids = preprocess_text(comment)
print("Comment: ", comment)
print("Tokenized Comment: ", tokenizer.convert_ids_to_tokens(input_ids[0])[0:20])
print("Token IDs: ", input_ids[0][0:20])

In [1]:
import pandas as pd

# Define the paths to your CSV files
csv_file_path_1 = 'data.csv'   # Replace with your actual first file path
csv_file_path_2 = 'AEcodiert240430_UTF8.csv'  # Replace with your actual second file path

# Read the first CSV file into a DataFrame
try:
    df1 = pd.read_csv(csv_file_path_1, delimiter=';', encoding='utf-8')
except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path_1}' not found.")
    exit()

# Read the second CSV file into a DataFrame
try:
    df2 = pd.read_csv(csv_file_path_2, delimiter=';', encoding='utf-8')
except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path_2}' not found.")
    exit()

# Display the first few rows of the concatenated DataFrame to verify
print("First few rows of the concatenated DataFrame:")
print(df1.head())  # Display the first few rows of the concatenated DataFrame

# Display the DataFrame to verify
print("DataFrame from CSV file:")
print(df2.head())

print("Columns in DataFrame:")
print(df1.columns)
print(df2.columns)

df = pd.DataFrame(df2)

import pandas as pd

def split_and_expand_df(df):
  """Splits rows with multiple llt_codes into separate rows, preserving descriptions.

  Args:
    df: The input DataFrame.

  Returns:
    The expanded DataFrame.
  """

  def split_row(row):
    llt_code_list = str(row['llt_code']).split(',')
    return pd.DataFrame({'llt_code': llt_code_list, 'ae_description': row['ae_description']})

  # Apply split_row to each row using apply
  df = df.apply(split_row, axis=1)

  # Concatenate resulting DataFrames
  df = pd.concat(df.tolist(), ignore_index=True)

  return df

splitData = pd.DataFrame(df)
expanded_df = split_and_expand_df(splitData.copy())

print(expanded_df)

# encode text 
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer


feature = expanded_df.llt_code
print('Feature: ')
print(feature)

# Create multiclass one-hot encoder
one_hot_multiclass = MultiLabelBinarizer()

# One-hot encode multiclass feature
one_hot_multiclass_encoded = one_hot_multiclass.fit_transform(feature)

print(one_hot_multiclass.classes_)

unique_labels = one_hot_multiclass.classes_
num_unique_labels = len(unique_labels)

print("Unique labels:", unique_labels)
print("Number of unique labels:", num_unique_labels)

# Add one-hot encoded multiclass feature to dataframe
expanded_df = expanded_df.join(pd.DataFrame(one_hot_multiclass_encoded, columns=one_hot_multiclass.classes_))

print(expanded_df)

#BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")
MAXLEN = 192

for text in expanded_df['ae_description']:
    print(tokenizer.tokenize(text))
    print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)))
#tokenize with tokenizer

# Define the tokenizer function
def tokenizer(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

# Apply the tokenizer function to the 'ae_description' column
expanded_df['ae_description'] = expanded_df['ae_description'].apply(tokenizer)

print(expanded_df)

# Tokenize the text data
tokenized_text = expanded_df['ae_description'].apply(tokenizer.tokenize)
print(tokenized_text)

# Convert the tokenized text sequences to numeric IDs
input_ids = padded_text.apply(lambda x: tokenizer.convert_tokens_to_ids(x))
print(input_ids)

# Model Architecture for multi class classification from ae_description
# Define the larger neural network model
vocabsize = len(tokenizer.word_index) + 1

model = Sequential([
    Dense(256, input_dim=vocabsize, activation='relu'),  # Increase the number of neurons
    BatchNormalization(),  # Add batch normalization
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_unique_labels, activation='sigmoid')  # Output dimension is the number of unique labels
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(expanded_df['ae_description'], expanded_df[one_hot_multiclass.classes_], test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Save the model
model.save('my_model.h5')



First few rows of the concatenated DataFrame:
   llt_code                   ae_description
0  10000001          Ventilationspneumonitis
1  10000002        11-Beta-Hydroxylasemangel
2  10000003      11-Oxysteroidaktivitaet erh
3  10000004  11-Oxysteroidaktivitaet erhoeht
4  10000005          17-Ketosteroide im Urin
DataFrame from CSV file:
             llt_code                                     ae_description
0            10022004                                   grippaler Infekt
1  10021434, 10022086  Impfreaktion nach COVID-19 Schutzimpfung - Sch...
2            10019045                                        Haarausfall
3            10000677                                 Akute Appendizitis
4  10022086, 10021434  Impfreaktion nach COVID-19 Impfung. Schmerzen ...
Columns in DataFrame:
Index(['llt_code', 'ae_description'], dtype='object')
Index(['llt_code', 'ae_description'], dtype='object')
       llt_code                                     ae_description
0      10022004         

/Users/tizander/Masterthesis/dl-code/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Feature: 
0        10022004
1        10021434
2        10022086
3        10019045
4        10000677
          ...    
1700     10084268
1701     10022086
1702     10025327
1703     10008531
1704     10016558
Name: llt_code, Length: 1705, dtype: object
[' ' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'n']
Unique labels: [' ' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'n']
Number of unique labels: 13
       llt_code                                     ae_description     0  1  \
0      10022004                                   grippaler Infekt  0  1  1   
1      10021434  Impfreaktion nach COVID-19 Schutzimpfung - Sch...  0  1  1   
2      10022086  Impfreaktion nach COVID-19 Schutzimpfung - Sch...  1  1  1   
3      10019045                                        Haarausfall  0  1  1   
4      10000677                                 Akute Appendizitis  0  1  1   
...         ...                                                ... .. .. ..   
1700   10084268                             

AttributeError: 'function' object has no attribute 'convert_tokens_to_ids'

In [ ]:
import nltk
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re

nltk_data_path = '/Users/tizander/Masterthesis/dl-code/myenv/lib/python3.12/site-packages/nltk'
os.environ["NLTK_DATA"] = nltk_data_path

# Set up stopwords and stemmer
stop_words = set(stopwords.words('german'))
stemmer = SnowballStemmer('german')

def preprocess_german_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize
    words = word_tokenize(text, language='german')
    # Remove stopwords and stem
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Preprocess 'ae_description' column
if 'ae_description' in expanded_df.columns:
    expanded_df['ae_description'] = expanded_df['ae_description'].apply(preprocess_german_text)
else:
    print("Error: 'ae_description' column not found in the DataFrame.")
    exit()

print("Text preprocessing completed successfully.")
print(expanded_df[['llt_code', 'ae_description']].head())


In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Tokenize the preprocessed text
tokenized_texts = expanded_df['ae_description'].apply(word_tokenize).tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Function to convert text to Word2Vec vectors
def text_to_vector(text):
    words = text.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(100)

# Convert all texts to vectors
text_vectors = np.array([text_to_vector(text) for text in expanded_df['ae_description']])


from sklearn.preprocessing import MultiLabelBinarizer

feature = expanded_df.llt_code
print('Feature: ')
print(feature)

# Create multiclass one-hot encoder
one_hot_multiclass = MultiLabelBinarizer()


# One-hot encode multiclass feature
one_hot_multiclass_encoded = one_hot_multiclass.fit_transform(feature)

print(one_hot_multiclass.classes_)

unique_labels = one_hot_multiclass.classes_
num_unique_labels = len(unique_labels)

print("Unique labels:", unique_labels)
print("Number of unique labels:", num_unique_labels)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Define the larger neural network model
model = Sequential([
    Dense(256, input_dim=100, activation='relu'),  # Increase the number of neurons
    BatchNormalization(),  # Add batch normalization
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_unique_labels, activation='sigmoid')  # Output dimension is the number of unique labels
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
model.fit(text_vectors, one_hot_multiclass_encoded, epochs=20, batch_size=32, validation_split=0.2)

print("Model training completed successfully.")
